### Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install librosa soundfile evaluate jiwer torchcodec "datasets>=3.4.1,<4.0.0"
!pip install webdataset

### You must login into the Huggingface (or create new account for free)

Publick huggingface dataset required for this demonstration

In [2]:
liepa_ds = "povilaskv/LT_whisper"


In [3]:
from unsloth import FastModel
from transformers import WhisperForConditionalGeneration
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "openai/whisper-large-v3-turbo",
    dtype = None, # Leave as None for auto detection
    load_in_4bit = False, # Set to True to do 4bit quantization which reduces memory
    auto_model = WhisperForConditionalGeneration,
    whisper_language = "Lithuanian",
    whisper_task = "transcribe",
    #token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2026.1.2: Fast Whisper patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.
unsloth/whisper-large-v3-turbo does not have a padding token! Will use pad_token = <|endoftext|>.


### Infererence the Vanilla model with the test sample

In [5]:
!wget https://github.com/PovilasKvedaras/STT_workshop/raw/main/povilas_lt.mp3

from IPython.display import Audio, display
display(Audio("povilas_lt.mp3", rate = 16000))

--2026-01-09 12:30:58--  https://github.com/PovilasKvedaras/STT_workshop/raw/main/povilas_lt.mp3
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PovilasKvedaras/STT_workshop/main/povilas_lt.mp3 [following]
--2026-01-09 12:30:58--  https://raw.githubusercontent.com/PovilasKvedaras/STT_workshop/main/povilas_lt.mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 369652 (361K) [audio/mpeg]
Saving to: ‘povilas_lt.mp3.1’

povilas_lt.mp3.1    100%[===================>] 360.99K  --.-KB/s    in 0.02s   

2026-01-09 12:30:59 (16.2 MB/s) - ‘povilas_lt.mp3.1’ saved [369652/369652]



In [6]:
from transformers import pipeline
import torch
FastModel.for_inference(model)
model.eval()
# Create pipeline without specifying the device
whisper = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=tokenizer.tokenizer,
    feature_extractor=tokenizer.feature_extractor,
    processor=tokenizer,
    return_language=True,
    #torch_dtype=torch.float16  # Remove the device parameter
    return_timestamps=True  # Add this parameter
)
# Example usage
audio_file = "povilas_lt.mp3"
transcribed_text = whisper(audio_file)


# Print the text
print(transcribed_text["text"])

# Print timestamps (word-level timestamps)
if "chunks" in transcribed_text:
    print("\nTimestamps:")
    for chunk in transcribed_text["chunks"]:
        print(f"Time: {chunk['timestamp']} - Text: {chunk['text']}")

Device set to use cuda:0
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 2

 Vis daugiau jaunų žmonių susidari su sanarius kausmais, kurie anksčiau buvo laikoma virsnių amdžiaus žmonių rūpeščio. Ilgos valandos prie kompiuterio, intensyvės treniruotės ar net stresas, visai tai gali sukelti nemalonius povečius sanariusse. Svarbu žinoti, kad skausmas neatsiranda be priežasties, tačiau yra gerų naujienų. Jį galima ne tik sumažinti, bet ir užkirsti kelią jo atsirodimui, rašomo eurovaistinės pranešimė žiniasklintai. Vis dažniau pastebime, kad sanarius kausmais skundžysi ne tik veresnio amžio žmonės, bet ir jaunesni. Asmenis pagrindinimis prižastimis tampa sėdimas gyvenimas būdas per mažas vizinio aktyvumas arba priešingai. Taigus per didelis krūvės, ypač sportuojant be atinkamo pasiruošimą, savo vadminį atlieka ir netinkamo laikysina. Antsvaris metybūs įpradžiai, pastaruoju metu ir stresas, kuris daro įteka visai raumenų ir kvalovų sistemai, sako vaistinė keminglė Kaminskaitė.

Timestamps:
Time: (0.58, 8.88) - Text:  Vis daugiau jaunų žmonių susidari su sanarius kau

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [7]:
model = FastModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    task_type = None, # ** MUST set this for Whisper **
)

Unsloth: Making `model.base_model.model.model.encoder` require gradients


In [8]:
model.print_trainable_parameters()

trainable params: 27,852,800 || all params: 836,730,880 || trainable%: 3.3288


In [9]:
import numpy as np
import tqdm
import io
import json
import librosa

model.generation_config.task = "transcribe"
model.config.suppress_tokens = []
model.generation_config.forced_decoder_ids = None
#model.generation_config.language = "<|lt|>"
model.generation_config.language = None  # Let model auto-detect



# Whisper timestamp tokens start at 50364 (<|0.00|>) and advance by 0.02s per +1.
WHISPER_TIMESTAMP_BASE_ID = 50364
WHISPER_TIMESTAMP_SECONDS_PER_ID = 0.02
WHISPER_TIMESTAMP_MAX_SECONDS = 30.0


def _whisper_timestamp_token_id(
    time_seconds: float,
    base_id: int = WHISPER_TIMESTAMP_BASE_ID,
    seconds_per_id: float = WHISPER_TIMESTAMP_SECONDS_PER_ID,
    max_seconds: float = WHISPER_TIMESTAMP_MAX_SECONDS,
) -> int:
    """Convert seconds -> Whisper timestamp token id with 0.02s rounding.

    Uses half-up rounding (not floor) to reduce systematic bias.
    """
    t = float(time_seconds)
    if t < 0.0:
        t = 0.0
    if t > max_seconds:
        t = max_seconds

    # Round to nearest 0.02s increment (half-up).
    steps = int((t / seconds_per_id) + 0.5)
    return base_id + steps


def _get_whisper_prefix_tokens(processor) -> list[int]:
    """Best-effort retrieval of Whisper BOS/LANG/TASK prefix tokens.

    Unsloth's `tokenizer` here behaves like a WhisperProcessor and exposes `.tokenizer`.
    We also remove `<|notimestamps|>` so the model can learn to emit timestamp tokens.
    """
    tok = getattr(processor, "tokenizer", processor)

    prefix: list[int] | None = None

    # Preferred: WhisperTokenizer{Fast}.prefix_tokens (already BOS/LANG/TASK/(maybe notimestamps)).
    if hasattr(tok, "prefix_tokens") and tok.prefix_tokens is not None:
        prefix = list(tok.prefix_tokens)

    # Fallback: try get_decoder_prompt_ids (some processor/tokenizer variants expose this).
    if prefix is None:
        get_ids = getattr(processor, "get_decoder_prompt_ids", None) or getattr(tok, "get_decoder_prompt_ids", None)
        if callable(get_ids):
            ids = None
            for lang in ("Lithuanian", "lt"):
                try:
                    ids = get_ids(language=lang, task="transcribe")
                    break
                except Exception:
                    continue

            if ids is not None:
                # HF commonly returns [(pos, id), ...]
                if len(ids) and isinstance(ids[0], (tuple, list)) and len(ids[0]) == 2:
                    prefix = [i for _, i in ids]
                else:
                    prefix = list(ids)

    if prefix is None:
        prefix = []

    # Ensure BOS at the beginning (required by the collator's BOS stripping logic).
    bos = getattr(tok, "bos_token_id", None)
    if bos is not None and (len(prefix) == 0 or prefix[0] != bos):
        prefix = [bos] + prefix

    # Remove the `<|notimestamps|>` token if present.
    notimestamps_id = None
    if hasattr(tok, "convert_tokens_to_ids"):
        notimestamps_id = tok.convert_tokens_to_ids("<|notimestamps|>")

    # Some tokenizers return `unk_token_id` when token is unknown; don't remove in that case.
    unk = getattr(tok, "unk_token_id", None)
    if notimestamps_id is not None and notimestamps_id != unk:
        prefix = [x for x in prefix if x != notimestamps_id]

    return prefix


def _build_timestamped_labels_for_sample(
    processor,
    text_list: list[str],
    end_time_stamps: list[float],
    max_label_length: int,
) -> list[int] | None:
    """Build labels as: prefix + (<|start|> text <|end|>)* + eos.

    `end_time_stamps` are cumulative sentence end times.
    Returns None if labels exceed `max_label_length`.
    """
    tok = processor.tokenizer
    prefix = _get_whisper_prefix_tokens(processor)

    labels: list[int] = list(prefix)
    prev_end = 0.0

    n = min(len(text_list), len(end_time_stamps))
    for j in range(n):
        end_t = float(end_time_stamps[j])
        start_t = prev_end

        # Guardrail for slightly non-monotonic stamps.
        if end_t < start_t:
            end_t = start_t

        start_id = _whisper_timestamp_token_id(start_t)
        end_id = _whisper_timestamp_token_id(end_t)

        # Tokenize sentence only (we manage BOS/EOS/prefix manually).
        sent_ids = tok(text_list[j], add_special_tokens=False).input_ids

        # Format: <|start|> Sentence <|end|>
        labels.append(start_id)
        labels.extend(sent_ids)
        labels.append(end_id)

        prev_end = end_t

        # Early reject if we already exceed the model's hard target limit.
        if len(labels) > max_label_length:
            return None

    labels.append(tok.eos_token_id)
    if len(labels) > max_label_length:
        return None

    return labels


def preprocess_batch(examples, expected_sampling_rate: int = 16000):
    """Batched preprocessing for streaming HF IterableDataset with columns: ogg/json.

    Per-row input schema:
      - ogg: {array: float32[time], sampling_rate: int, path: str}
      - json:
          - text: str
          - text_list: list[str] (optional)
          - duration_stamps: list[float] cumulative sentence end times (optional)

    Per-row output schema:
      - input_features: Whisper log-mel features
      - labels: token ids (unpadded; collator pads)

    If timestamp fields are present, labels are built with real Whisper timestamp tokens.
    """
    ogg_list = examples["ogg"]
    json_list = examples["json"]

    audio_arrays = [x["array"] for x in ogg_list]
    sampling_rates = [x.get("sampling_rate", expected_sampling_rate) for x in ogg_list]

    # Your data should already be 16kHz; this is a guardrail.
    if any(sr != expected_sampling_rate for sr in sampling_rates):
        raise ValueError(
            f"Found non-{expected_sampling_rate}Hz audio in batch: {sorted(set(sampling_rates))}"
        )

    features = tokenizer.feature_extractor(audio_arrays, sampling_rate=expected_sampling_rate)

    # Filter out individual samples where token length exceeds Whisper's max (448 tokens)
    # This drops only the offending samples, NOT the entire batch
    max_label_length = 447  # Whisper max is 448, use 447 for safety margin

    labels_out: list[list[int]] = []
    valid_indices: list[int] = []

    for i, meta in enumerate(json_list):
        # Prefer timestamp-aware training data when present.
        text_list = meta.get("text_list")
        end_stamps = meta.get("duration_stamps")

        ids: list[int] | None = None

        if isinstance(text_list, list) and isinstance(end_stamps, list) and len(text_list) > 0 and len(end_stamps) > 0:
            ids = _build_timestamped_labels_for_sample(
                tokenizer,
                text_list=text_list,
                end_time_stamps=end_stamps,
                max_label_length=max_label_length,
            )
        else:
            # Fallback to your original behavior (no timestamps) if fields are missing.
            txt = meta.get("text", "")
            ids = tokenizer.tokenizer(txt)["input_ids"]

        if ids is None:
            continue

        if len(ids) <= max_label_length:
            valid_indices.append(i)
            labels_out.append(ids)

    # Return only valid samples (empty lists if none are valid)
    return {
        "input_features": [features["input_features"][i] for i in valid_indices],
        "labels": labels_out,
    }


def debug_print_labels_with_timestamps(batch_or_iterable, n: int = 5):
    """Print decoded labels to verify Whisper timestamp tokens are present.

    Usage examples:
      - After calling preprocess_batch(...): debug_print_labels_with_timestamps(out, n=3)
      - On an iterable dataset: debug_print_labels_with_timestamps(train_iterable, n=3)

    Note: uses skip_special_tokens=False so `<|0.00|>` etc remain visible.
    """
    import itertools

    tok = tokenizer.tokenizer

    def _print_one(idx: int, ids: list[int]):
        print(f"--- sample {idx} (len={len(ids)}) ---")
        print(tok.decode(ids, skip_special_tokens=False))

        ts_ids = [x for x in ids if WHISPER_TIMESTAMP_BASE_ID <= x <= (WHISPER_TIMESTAMP_BASE_ID + int(WHISPER_TIMESTAMP_MAX_SECONDS / WHISPER_TIMESTAMP_SECONDS_PER_ID))]
        if ts_ids:
            ts_secs = [round((x - WHISPER_TIMESTAMP_BASE_ID) * WHISPER_TIMESTAMP_SECONDS_PER_ID, 2) for x in ts_ids]
            print("timestamp_seconds:", ts_secs[:25], "..." if len(ts_secs) > 25 else "")

    if isinstance(batch_or_iterable, dict) and "labels" in batch_or_iterable:
        for i, ids in enumerate(batch_or_iterable["labels"][:n]):
            _print_one(i, ids)
    else:
        for i, ex in enumerate(itertools.islice(batch_or_iterable, n)):
            _print_one(i, ex["labels"])



In [10]:
# Timestamp-aware label construction for Whisper (adds real <|t|> tokens)
# NOTE: This cell intentionally re-defines `preprocess_batch` used below in `.map(...)`.

from typing import List, Optional

# Whisper timestamp tokens advance by 0.02s per +1.
# IMPORTANT: Some tokenizer variants shift the base ID, so we resolve `<|0.00|>` from the tokenizer.
WHISPER_TIMESTAMP_SECONDS_PER_ID = 0.02
WHISPER_TIMESTAMP_MAX_SECONDS = 30.0


def _resolve_whisper_timestamp_base_id(tok, fallback: int = 50364) -> int:
    try:
        base = tok.convert_tokens_to_ids("<|0.00|>") if hasattr(tok, "convert_tokens_to_ids") else None
    except Exception:
        base = None

    # Some implementations may return a list; normalize.
    if isinstance(base, (list, tuple)):
        base = base[0] if len(base) else None

    if isinstance(base, int) and base > 0:
        return base

    return fallback


WHISPER_TIMESTAMP_BASE_ID = _resolve_whisper_timestamp_base_id(tokenizer.tokenizer)


def whisper_time_to_token_id(
    time_seconds: float,
    base_id: int = WHISPER_TIMESTAMP_BASE_ID,
    seconds_per_id: float = WHISPER_TIMESTAMP_SECONDS_PER_ID,
    max_seconds: float = WHISPER_TIMESTAMP_MAX_SECONDS,
) -> int:
    """Convert seconds -> Whisper timestamp token id.

    - Clamps to [0, 30] seconds
    - Rounds to nearest 0.02s increment (half-up)
    """
    t = float(time_seconds)
    if t < 0.0:
        t = 0.0
    if t > max_seconds:
        t = max_seconds

    steps = int((t / seconds_per_id) + 0.5)  # half-up
    return base_id + steps


def get_whisper_prefix_tokens(processor) -> List[int]:
    """Get BOS/LANG/TASK prefix tokens and remove `<|notimestamps|>`.

    Unsloth's `tokenizer` in this notebook acts like a WhisperProcessor.
    """
    tok = getattr(processor, "tokenizer", processor)

    prefix: Optional[List[int]] = None

    # Preferred: prefix_tokens already contains BOS + language + task (+ maybe notimestamps)
    if hasattr(tok, "prefix_tokens") and tok.prefix_tokens is not None:
        prefix = list(tok.prefix_tokens)

    # Fallback: try processor/tokenizer `get_decoder_prompt_ids`
    if prefix is None:
        get_ids = getattr(processor, "get_decoder_prompt_ids", None) or getattr(tok, "get_decoder_prompt_ids", None)
        if callable(get_ids):
            ids = None
            for lang in ("Lithuanian", "lt"):
                try:
                    ids = get_ids(language=lang, task="transcribe")
                    break
                except Exception:
                    continue

            if ids is not None:
                # HF often returns [(pos, id), ...]
                if len(ids) and isinstance(ids[0], (tuple, list)) and len(ids[0]) == 2:
                    prefix = [i for _, i in ids]
                else:
                    prefix = list(ids)

    if prefix is None:
        prefix = []

    # Ensure BOS at the beginning (also helps the collator's BOS stripping logic)
    bos = getattr(tok, "bos_token_id", None)
    if bos is not None and (len(prefix) == 0 or prefix[0] != bos):
        prefix = [bos] + prefix

    # Remove `<|notimestamps|>` so timestamps are learnable.
    # Some Whisper tokenizer variants behave oddly with `convert_tokens_to_ids`, so we try a few ways.
    remove_ids = set()

    if hasattr(tok, "convert_tokens_to_ids"):
        try:
            remove_ids.add(tok.convert_tokens_to_ids("<|notimestamps|>"))
        except Exception:
            pass

    # Encoding the special token as text is often the most reliable.
    try:
        enc = tok.encode("<|notimestamps|>", add_special_tokens=False)
        if isinstance(enc, list):
            remove_ids.update(enc)
    except Exception:
        pass

    # Drop unknown/None ids
    unk = getattr(tok, "unk_token_id", None)
    remove_ids = {i for i in remove_ids if i is not None and i != unk}

    if remove_ids:
        prefix = [x for x in prefix if x not in remove_ids]

    return prefix


def build_timestamped_labels(
    processor,
    text_list: List[str],
    end_time_stamps: List[float],
    max_label_length: int,
) -> Optional[List[int]]:
    """Build labels: prefix + (<|start|> text <|end|>)* + eos.

    `end_time_stamps` are cumulative sentence end times.
    Returns None if labels exceed `max_label_length`.
    """
    tok = processor.tokenizer
    labels: List[int] = list(get_whisper_prefix_tokens(processor))

    prev_end = 0.0
    n = min(len(text_list), len(end_time_stamps))

    for j in range(n):
        end_t = float(end_time_stamps[j])
        start_t = prev_end

        # Guardrail for slightly non-monotonic stamps
        if end_t < start_t:
            end_t = start_t

        start_id = whisper_time_to_token_id(start_t)
        end_id = whisper_time_to_token_id(end_t)

        sent_ids = tok(text_list[j], add_special_tokens=False).input_ids

        labels.append(start_id)
        labels.extend(sent_ids)
        labels.append(end_id)

        prev_end = end_t

        if len(labels) > max_label_length:
            return None

    labels.append(tok.eos_token_id)
    if len(labels) > max_label_length:
        return None

    return labels


def preprocess_batch(examples, expected_sampling_rate: int = 16000):
    """Batched preprocessing for streaming HF IterableDataset with columns: ogg/json.

    Per-row input schema:
      - ogg: {array: float32[time], sampling_rate: int, path: str}
      - json:
          - text: str
          - text_list: list[str] (optional)
          - duration_stamps: list[float] cumulative sentence end times (optional)

    Per-row output schema:
      - input_features: Whisper log-mel features
      - labels: token ids (unpadded; collator pads)

    If timestamp fields are present, labels are built with real Whisper timestamp tokens.
    """
    ogg_list = examples["ogg"]
    json_list = examples["json"]

    audio_arrays = [x["array"] for x in ogg_list]

    # `datasets` may provide audio as dicts (streaming) or AudioDecoder objects (arrow datasets).
    # AudioDecoder supports `x["sampling_rate"]` but not `x.get(...)`.
    def _audio_sampling_rate(x, default: int) -> int:
        try:
            return x.get("sampling_rate", default)
        except AttributeError:
            try:
                return x["sampling_rate"]
            except Exception:
                return default

    sampling_rates = [_audio_sampling_rate(x, expected_sampling_rate) for x in ogg_list]

    # Your data should already be 16kHz; this is a guardrail.
    if any(sr != expected_sampling_rate for sr in sampling_rates):
        raise ValueError(
            f"Found non-{expected_sampling_rate}Hz audio in batch: {sorted(set(sampling_rates))}"
        )

    features = tokenizer.feature_extractor(audio_arrays, sampling_rate=expected_sampling_rate)

    # Filter out individual samples where token length exceeds Whisper's max (448 tokens)
    # This drops only the offending samples, NOT the entire batch
    max_label_length = 447  # Whisper max is 448, use 447 for safety margin

    labels_out: List[List[int]] = []
    valid_indices: List[int] = []

    for i, meta in enumerate(json_list):
        text_list = meta.get("text_list")
        end_stamps = meta.get("duration_stamps")

        ids: Optional[List[int]] = None

        if isinstance(text_list, list) and isinstance(end_stamps, list) and len(text_list) > 0 and len(end_stamps) > 0:
            ids = build_timestamped_labels(
                tokenizer,
                text_list=text_list,
                end_time_stamps=end_stamps,
                max_label_length=max_label_length,
            )
        else:
            # Fallback to original behavior (no timestamps) if fields are missing.
            txt = meta.get("text", "")
            ids = tokenizer.tokenizer(txt)["input_ids"]

        if ids is None:
            continue

        if len(ids) <= max_label_length:
            valid_indices.append(i)
            labels_out.append(ids)

    return {
        "input_features": [features["input_features"][i] for i in valid_indices],
        "labels": labels_out,
    }


def debug_print_labels_with_timestamps(batch_or_iterable, n: int = 5):
    """Decode & print first N label sequences to verify timestamp tokens.

    - Uses `skip_special_tokens=False` so `<|0.00|>` etc remain visible.
    - Accepts either:
      - the dict returned by `preprocess_batch(...)`, or
      - an iterable dataset yielding dicts with `labels`.
    """
    import itertools

    tok = tokenizer.tokenizer
    max_ts_id = WHISPER_TIMESTAMP_BASE_ID + int(WHISPER_TIMESTAMP_MAX_SECONDS / WHISPER_TIMESTAMP_SECONDS_PER_ID)

    def _print_one(idx: int, ids: List[int]):
        print(f"--- sample {idx} (len={len(ids)}) ---")

        # Some Whisper tokenizers only render `<|t|>` tokens when `decode_with_timestamps=True`.
        try:
            decoded = tok.decode(ids, skip_special_tokens=False, decode_with_timestamps=True)
        except TypeError:
            decoded = tok.decode(ids, skip_special_tokens=False)
        print(decoded)

        ts_ids = [x for x in ids if WHISPER_TIMESTAMP_BASE_ID <= x <= max_ts_id]
        if ts_ids:
            ts_secs = [round((x - WHISPER_TIMESTAMP_BASE_ID) * WHISPER_TIMESTAMP_SECONDS_PER_ID, 2) for x in ts_ids]
            ts_tokens = [tok.convert_ids_to_tokens(x) for x in ts_ids[:25]]
            print("timestamp_tokens:", ts_tokens[:25], "..." if len(ts_ids) > 25 else "")
            print("timestamp_seconds:", ts_secs[:25], "..." if len(ts_secs) > 25 else "")

    if isinstance(batch_or_iterable, dict) and "labels" in batch_or_iterable:
        for i, ids in enumerate(batch_or_iterable["labels"][:n]):
            _print_one(i, ids)
    else:
        for i, ex in enumerate(itertools.islice(batch_or_iterable, n)):
            _print_one(i, ex["labels"])


In [11]:
import webdataset as wds
from huggingface_hub import get_token
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from torch.utils.data import IterableDataset



eval_liepa = load_dataset(liepa_ds, split="test")
#eval_cb = load_dataset(cb_ds, split="test")

eval = eval_liepa

#eval = concatenate_datasets([eval_liepa, eval_cb])


class RepeatN(IterableDataset):
    def __init__(self, make_iterable, n: int):
        self.make_iterable = make_iterable
        self.n = n

    def __iter__(self):
        for _ in range(self.n):
            yield from iter(self.make_iterable())

# Preprocessing batch size is CPU-side; keep moderate for memory stability.
preprocess_batch_size = 64

def make_train_iterable():
    train_liepa = load_dataset(liepa_ds, streaming=True, split="train")
    #train_cb = load_dataset(cb_ds, streaming=True, split="train")
    #base = concatenate_datasets([train_liepa, train_cb])
    base = train_liepa
    # shuffle the dataset
    base = base.shuffle(seed=42)

    return base.map(
        preprocess_batch,
        batched=True,
        batch_size=preprocess_batch_size,
        remove_columns=base.column_names,
    )

train_iterable = RepeatN(make_train_iterable, n=3)

# Eval is preprocessed the same way
eval_dataset = eval.map(
    preprocess_batch,
    batched=True,
    batch_size=preprocess_batch_size,
    remove_columns=eval.column_names,
)



In [12]:
tmp = preprocess_batch(next(iter(load_dataset(liepa_ds, streaming=True, split="train").batch(8))))
debug_print_labels_with_timestamps(tmp, n=3)

--- sample 0 (len=189) ---
<|endoftext|><|startoftranscript|><|lt|><|transcribe|><|0.00|>Kai įstengdavau pataisyti.<|2.28|><|2.28|>Ji būdavo tokia ištrūškusi.<|4.46|><|4.46|>Iš savo pavaldinių.<|6.36|><|6.36|>Ar galiu įsipilti?<|8.16|><|8.16|>Kai kurie vedė.<|9.92|><|9.92|>Paskuo nubėgo pas motiną.<|11.96|><|11.96|>Intonavimas.<|13.60|><|13.60|>Kur gi buvai? Kur gi tu išbėgai, mus palikęs?<|17.52|><|17.52|>Su kuriais galėsime bendrauti savo kalba.<|20.32|><|20.32|>Atsiprašau, kažkokia meška iš laužo.<|22.50|><|22.50|>Išgirdau raginimus.<|24.54|><|24.54|>Belieka paminėti dar vieną dalyką.<|26.52|><|endoftext|>
timestamp_tokens: ['<|0.00|>', '<|2.28|>', '<|2.28|>', '<|4.46|>', '<|4.46|>', '<|6.36|>', '<|6.36|>', '<|8.16|>', '<|8.16|>', '<|9.92|>', '<|9.92|>', '<|11.96|>', '<|11.96|>', '<|13.60|>', '<|13.60|>', '<|17.52|>', '<|17.52|>', '<|20.32|>', '<|20.32|>', '<|22.50|>', '<|22.50|>', '<|24.54|>', '<|24.54|>', '<|26.52|>'] 
timestamp_seconds: [0.0, 2.28, 2.28, 4.46, 4.46, 6.36, 6.36, 8

In [13]:
# @title Create compute_metrics and datacollator
import evaluate
import torch
import numpy as np
from dataclasses import dataclass
from typing import Any, Dict, List, Union


metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


<a name="Train"></a>
### Train the model
Now let's use Huggingface  `Seq2SeqTrainer`! More docs here: [Transformers docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [14]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from unsloth import is_bf16_supported

# Whisper decoder has a hard limit (typically 448) on target tokens.
# If a single sample exceeds this, padding would blow up the whole batch.
max_label_length = getattr(model.config, "max_target_positions", None)

trainer = Seq2SeqTrainer(
    model = model,
    train_dataset = train_iterable,
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=tokenizer),
    eval_dataset = eval_dataset,
    #compute_metrics=compute_metrics,
    tokenizer = tokenizer.feature_extractor,
    args = Seq2SeqTrainingArguments(
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        per_device_train_batch_size = 12,
        gradient_accumulation_steps = 2,
        warmup_steps = 10,
        max_steps = 90,
        learning_rate = 2e-4,
        optim = "adamw_8bit",
        bf16 = is_bf16_supported(),  # Use bf16 if supported, else use fp16
        fp16 = not is_bf16_supported(),
        weight_decay = 0.01, # penalise large weights
        generation_max_length=256,
        remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
        lr_scheduler_type = "linear",
        label_names = ['labels'],
        logging_steps=1,
        eval_strategy="steps",
        eval_steps = 10,
        seed = 3407,
        do_eval=True,
        output_dir = "outputs",
        report_to = "none"
    ),
)

/tmp/ipython-input-2818120897.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,160 | Num Epochs = 9,223,372,036,854,775,807 | Total steps = 90
O^O/ \_/ \    Batch size per device = 12 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (12 x 2 x 1) = 24
 "-____-"     Trainable parameters = 27,852,800 of 836,730,880 (3.33% trained)
Unsloth: Not an error, but WhisperForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss
10,1.674200,1.369149
20,0.830800,0.844559
30,0.795500,0.775711
40,0.772000,0.723538
50,0.648300,0.699755
60,0.514300,0.690858
70,0.531600,0.672435
80,0.491500,0.660780
90,0.494400,0.651215


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [16]:
#model.save_pretrained("lora_model")  # Local saving
#tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

# Test model post fine-tuning result

In [17]:
FastModel.for_inference(model)
model.eval()
# Create pipeline without specifying the device
whisper = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=tokenizer.tokenizer,
    feature_extractor=tokenizer.feature_extractor,
    processor=tokenizer,
    return_language=True,
    #torch_dtype=torch.float16  # Remove the device parameter
    return_timestamps=True  # Add this parameter
)
# Example usage
#audio_file = "povilas_lt.mp3"
transcribed_text = whisper(audio_file)


# Print the text
print(transcribed_text["text"])

# Print timestamps (word-level timestamps)
if "chunks" in transcribed_text:
    print("\nTimestamps:")
    for chunk in transcribed_text["chunks"]:
        print(f"Time: {chunk['timestamp']} - Text: {chunk['text']}")

Device set to use cuda:0


Vis daugiau jaunų žmonių susidarė su sanarius skausmais, kurie anksčiau buvo laikoma viresnio amžio žmonių rūpėščiu.Ilgos valandos prie kompiuterio, intensyvės treniruotės ar net stresas, visai tai gali sukelti nemalonius poveučius sanariuose.Svarbu žinoti, kad skausmas neatsiranda be priežasties, tačiau yra gerų naujienų.Jį galima ne tik sumažinti, bet ir užkirsti kelią jo atsirodimui, rašoma.Vis dažniau pastebėme, kad senarių skausmais skundžiusi ne tik veresnio amžio žmonės, bet ir jaunesni asmenis.Pagrindinis priežasdamis tampa sėdimas gyvenimas būdas per mažas vizinių aktyvumas arba priešiangai, staigus per didelis krūvius, ypač sportuojant be atinkamą pasiruošimą.Savo vadinį lieka ir netinkamą laikysiną. Antsvaris, metybūs įpračiai, pastaruoju metu ir stresas, kuris daro įtiką visai romių. sako vaistininkė mingilė Kaminskaitė.

Timestamps:
Time: (0.0, 7.0) - Text: Vis daugiau jaunų žmonių susidarė su sanarius skausmais, kurie anksčiau buvo laikoma viresnio amžio žmonių rūpėščiu.
